In [4]:
import numpy as np
import pandas as pd
import sys
from openai import OpenAI
import os
import cv2
import re
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import base64
from tqdm import tqdm
from dotenv import load_dotenv
import re
from groq import Groq

In [5]:
df = pd.read_csv('Data/New230SHEETS.csv')
df

,id,image_name,ocr_txt
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...
...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n..."
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah..."
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...


In [6]:
# Check for missing values in the 'image_name' column
missing_image_names = df['image_name'].isna()

# Show rows where 'image_name' is missing
df_missing = df[missing_image_names]
print(df_missing)


Empty DataFrame
Columns: [id, image_name, ocr_txt]
Index: []


In [7]:
# Drop rows where 'image_name' is missing
df = df.dropna(subset=['ocr_txt'])

# Optionally, reset the index if you want a clean index after dropping
df = df.reset_index(drop=True)

In [8]:
# Load prompt template
with open('Prompt/interpreter.md', 'r', encoding='utf-8') as f:
    prompt_template = f.read()

In [9]:
# Test with first meme
test_row = df.iloc[0]
image_text = test_row['ocr_txt']
image_name = test_row['image_name']

# Replace placeholders
final_prompt = prompt_template.replace('{meme_text}', str(image_text))
final_prompt = final_prompt.replace('{image_path}', image_name)

# Show the final prompt
print("FINAL PROMPT THAT WILL BE SENT TO OPENAI:")
print("="*60)
print(final_prompt)
print("="*60)

FINAL PROMPT THAT WILL BE SENT TO OPENAI:
# Role
You are an expert in interpreting meme since you are a master of analyzing the hidden message behind meme in Indonesian language

# Task
Your task is to generate a hidden message or the context of the meme

## Things Need to be Considered
- There are 2 inputs which are meme image and meme text. Please, refer to those input for interpret meme
- All of the memes are in Indonesian language. Therefore, please interpret meme in Indonesian language
- The connection between meme image and meme text can be metaporical. Therefore, analyze both of those input carefully (i.e., a picture of a grumpy man is represent bad stuff and dislike)

## Example

### Example 1
Input Image = meme_data_999.jpg
Input Text = Sudah Kubilang jangan lawan King Indonesia
Output = Tim sepak bola Indonesia adalah yang terbaik saat ini dan susah untuk dikalahkan. Oleh karena itu jangan coba-coba melawan tim sepak bola Indonesia

## Query
Input Image = Meme_Data_1.jpg
Inpu

In [10]:
def encode_image_to_base64(image_path):
    """Encode image to base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [12]:
# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from environment
client = Groq(
    api_key=os.getenv('GROQ_API_KEY')
)

# Verify API key is loaded
if not os.getenv('GROQ_API_KEY'):
    raise ValueError("GROQ_API_KEY not found in environment variables. Please create a .env file with your API key.")

def interpret_meme(row):
    """Interpret meme using OpenAI Vision API"""
    # Get image path and text from your CSV structure
    image_path = f"Data/Filtered 230/{row['image_name']}"
    image_text = row['ocr_txt']
    
    # Encode image
    base64_image = encode_image_to_base64(image_path)
    
    # Create prompt by replacing placeholders
    prompt = prompt_template.replace('{meme_text}', str(image_text))
    prompt = prompt.replace('{image_path}', row['image_name'])
    
    try:
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            max_tokens=2048,
            temperature=0
        )

        result = response.choices[0].message.content.strip()
        return result
    except Exception as e:
        print("Error:", e)
        return None

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [15]:
print("GROQ_API_KEY:", os.getenv('GROQ_API_KEY'))


GROQ_API_KEY: None


In [42]:
sample_row = df.iloc[0]  # Use the first row for testing
output = interpret_meme(sample_row)
print(output)

**Pesan tersembunyi atau konteks meme:**

Meme ini menyindir cowok-cowok zaman sekarang yang ingin punya pacar cantik, tapi tidak mau mengeluarkan uang untuk perawatan pacarnya di salon. Ketika dimintai uang untuk perawatan, mereka malah menuduh pacarnya matre (materialistis). Padahal, perawatan di salon itu memang mahal dan tidak bisa dibayar dengan BPJS (asuransi kesehatan pemerintah yang biasanya hanya untuk kebutuhan medis, bukan kecantikan). Meme ini menekankan bahwa kalau mau punya pacar cantik, harus siap juga dengan konsekuensi biaya perawatannya, bukan malah mengeluh atau menuduh matre.


In [43]:
# Process all memes
results = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing memes"):
    result = interpret_meme(row)
    results.append({
        'id': row['id'],
        'image_name': row['image_name'],
        'ocr_txt': row['ocr_txt'],
        'full_response': result
    })
    
    # Optional: Add delay to respect rate limits
    import time
    time.sleep(1)

results

Processing memes: 100%|██████████| 230/230 [20:11<00:00,  5.27s/it]


[{'id': 1,
  'image_name': 'Meme_Data_1.jpg',
  'ocr_txt': 'COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK GILIRAN DIMINTAIN DUIT SALON, BILANGNYA MATRE EMANG LO KIRA PERAWATAN DI SALON PAKE BPJS??',
  'full_response': 'Output = Meme ini menyindir cowok-cowok zaman sekarang yang ingin punya pacar cantik, tapi tidak mau mengeluarkan uang untuk perawatan pacarnya di salon. Padahal, untuk tampil cantik butuh biaya, dan tidak bisa gratis seperti layanan BPJS. Jadi, meme ini menegaskan bahwa kalau mau punya pacar cantik, harus siap juga dengan konsekuensi biaya perawatannya, jangan malah menuduh matre.'},
 {'id': 2,
  'image_name': 'Meme_Data_2.jpg',
  'ocr_txt': 'ayat suci dalam bio bukan jaminan kalo\n dia orang yang baik',
  'full_response': 'Output = Meme ini menyampaikan pesan bahwa menampilkan ayat suci di bio media sosial seseorang tidak otomatis membuat orang tersebut benar-benar baik dalam kehidupan nyata. Seringkali, orang menggunakan simbol-simbol religius hanya untuk pencitraan,

In [117]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df

,id,image_name,ocr_txt,full_response
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Output = Meme ini menyindir cowok-cowok zaman ...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Output = Meme ini menyampaikan pesan bahwa men...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini membandingkan tingkat kematangan dagi...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Output = Meme ini menggunakan permainan kata y...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...
...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Output = Meme ini menyindir perbandingan antar...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...


In [118]:
def extract_combined_robust(response):
    # Try to extract after 'output ='
    match_output = re.search(r'output\s*=\s*([\s\S]*)', response, re.IGNORECASE)
    if match_output:
        return match_output.group(1).strip()
    # Try to extract after 'Pesan tersembunyi dari meme ini adalah:'
    match_pesan = re.search(r'Pesan tersembunyi dari meme ini adalah:\s*([\s\S]*)', response, re.IGNORECASE)
    if match_pesan:
        return match_pesan.group(1).strip()
    # Fallback to original response
    return response.strip()

results_df['output_extracted'] = results_df['full_response'].apply(lambda x: extract_combined_robust(str(x)))

In [119]:
# Example usage:
response = results_df['output_extracted'][141]
print(extract_combined_robust(response))

Meme ini menyindir laki-laki yang suka mengeluh atau memprotes pacarnya karena tidak bisa memasak, digambarkan dengan wajah yang terlihat marah atau tidak dewasa. Sementara itu, laki-laki yang bisa memasak sendiri digambarkan dengan sosok yang lebih keren dan dewasa. Intinya, laki-laki yang bisa memasak sendiri dianggap lebih keren dan dewasa daripada yang hanya bisa mengeluh ke pasangannya.


In [121]:
results_df

,id,image_name,ocr_txt,full_response,output_extracted
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,Output = Meme ini menyindir cowok-cowok zaman ...,Meme ini menyindir cowok-cowok zaman sekarang ...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Output = Meme ini menyampaikan pesan bahwa men...,Meme ini menyampaikan pesan bahwa menampilkan ...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini membandingkan tingkat kematangan dagi...,"Rendang, makanan khas Indonesia (khususnya Min..."
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Output = Meme ini menggunakan permainan kata y...,Meme ini menggunakan permainan kata yang lucu....
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,Pesan tersembunyi dari meme ini adalah sindira...,Pesan tersembunyi dari meme ini adalah sindira...
...,...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Pesan tersembunyi dari meme ini adalah sindira...,Pesan tersembunyi dari meme ini adalah sindira...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Pesan tersembunyi dari meme ini adalah tentang...,Pesan tersembunyi dari meme ini adalah tentang...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Output = Meme ini menyindir perbandingan antar...,Meme ini menyindir perbandingan antara Singapo...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Pesan tersembunyi dari meme ini adalah tentang...,Pesan tersembunyi dari meme ini adalah tentang...


In [123]:
# Save results
results_df.to_csv('gpt4.1_results.csv', index=False)